**INSTALL LIBRARY**

In [1]:
!pip install --upgrade gensim

In [2]:
!pip install pyldavis==3.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.2.1-py2.py3-none-any.whl size=136164 sha256=6e8ca20db868a4226b55f6ca0c1419751c5eae0c21aae2e582fd66ff58c601d9
  Stored in directory: /root/.cache/pip/wheels/d3/35/9c/c084bb1c3a4b8a7feb32d57c88f572dcf5c3accc949478893f
Successfully built pyldavis


In [3]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.0 MB/s eta 0:00:00


In [4]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=060c3d7343ac7d8cfb93d3c8d4db8f6d8fb1431bdf7f99fe4a317777ca488daa
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


**MENYIAPKAN DATA**

In [5]:
import pandas as pd
import numpy as np
import nltk

In [90]:
from google.colab import files
uploades = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving SI-UNUSA.xlsx to SI-UNUSA.xlsx


In [91]:
DATA_SI = pd.read_excel("SI-UNUSA.xlsx")
DATA_SI.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Dokumen,Abstrak
0,d1,"Sampai dengan 30 Oktober 2019, total jumlah pe..."
1,d2,"Pada tubuh manusia, ginjal merupakan organ yan..."
2,d3,Taman Hutan Raya Raden Soerjo (TAHURA R.Soerjo...
3,d4,Taman Hutan Raya Raden Soerjo (TAHURA R.Soerjo...
4,d5,Ginjal terdiri dari beberapa bagian antara lai...


**PREPROCESSING**

In [92]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].str.lower()

print('Case Folding Result : \n')
print(DATA_SI['Abstrak'].head(5))

Case Folding Result : 

0    sampai dengan 30 oktober 2019, total jumlah pe...
1    pada tubuh manusia, ginjal merupakan organ yan...
2    taman hutan raya raden soerjo (tahura r.soerjo...
3    taman hutan raya raden soerjo (tahura r.soerjo...
4    ginjal terdiri dari beberapa bagian antara lai...
Name: Abstrak, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [93]:
import string
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [94]:
# ------ Tokenizing ---------

nltk.download('punkt')

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

DATA_SI['Abstrak'] = DATA_SI['Abstrak'].apply(remove_singl_char)

# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

DATA_SI['Abstrak_tokens'] = DATA_SI['Abstrak'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(DATA_SI['Abstrak_tokens'].head())

Tokenizing Result : 

0    [sampai, dengan, oktober, total, jumlah, penye...
1    [pada, tubuh, manusia, ginjal, merupakan, orga...
2    [taman, hutan, raya, raden, soerjo, tahura, rs...
3    [taman, hutan, raya, raden, soerjo, tahura, rs...
4    [ginjal, terdiri, dari, beberapa, bagian, anta...
Name: Abstrak_tokens, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:11: DeprecationWarning: invalid escape sequence '\w'
<>:37: DeprecationWarning: invalid escape sequence '\s'
<>:11: DeprecationWarning: invalid escape sequence '\w'
<>:37: DeprecationWarning: invalid escape sequence '\s'
<ipython-input-94-f56c0c34adfd>:11: DeprecationWarning: invalid escape sequence '\w'
  text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
<ipython-input-94-f56c0c34adfd>:37: DeprecationWarning: invalid escape sequence '\s'
  return re.sub('\s+',' ',text)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [95]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

DATA_SI['Abstrak_tokens_fdist'] = DATA_SI['Abstrak_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
print(DATA_SI['Abstrak_tokens_fdist'].head().apply(lambda x : x.most_common()))


Frequency Tokens : 

0    [(ewallet, 7), (pembayaran, 6), (untuk, 6), (y...
1    [(dan, 7), (fitur, 7), (glomerulus, 7), (tubul...
2    [(quality, 11), (dengan, 10), (memiliki, 9), (...
3    [(quality, 11), (dengan, 10), (memiliki, 9), (...
4    [(ginjal, 11), (dari, 9), (data, 7), (adalah, ...
Name: Abstrak_tokens_fdist, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [96]:
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [97]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'pt', 'hasil' 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'saya', 'yang', 'dan', 'aja', 'biar', 'bikin', 'nan', 'yah', 'tuh', 'nih', 'sih', 'dan', 'jika', 'itu', 'untuk', 'ke', 'pada', 'terus', 'saya'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

DATA_SI['Abstrak_tokens_WSW'] = DATA_SI['Abstrak_tokens'].apply(stopwords_removal)

print(DATA_SI['Abstrak_tokens_WSW'].head())

0    [oktober, total, penyelenggara, fintech, pendu...
1    [tubuh, manusia, ginjal, organ, berfungsi, fil...
2    [taman, hutan, raya, raden, soerjo, tahura, rs...
3    [taman, hutan, raya, raden, soerjo, tahura, rs...
4    [ginjal, korteks, ginjal, medula, ginjal, pelv...
Name: Abstrak_tokens_WSW, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**NORMALISASI**

In [98]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in DATA_SI['Abstrak_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))


2329


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [99]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

    # untuk melihat hasilnya silahkan jalankan baris di bawah ini
    print(term,":" ,term_dict[term])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


oktober : oktober
total : total
penyelenggara : selenggara
fintech : fintech
pendukung : dukung
sistem : sistem
pembayaran : bayar
diawasi : awas
langsung : langsung
bank : bank
indonesia : indonesia
perusahaan : usaha
salah : salah
jenis : jenis
payment : payment
settlement : settlement
and : and
clearing : clearing
ranah : ranah
contohnya : contoh
ewallet : ewallet
aplikasi : aplikasi
mempengaruhi : pengaruh
pola : pola
hidup : hidup
mahasiswa : mahasiswa
universitas : universitas
nahdlatul : nahdlatul
ulama : ulama
surabaya : surabaya
transaksi : transaksi
penawaranpenawaran : penawaranpenawaran
instan : instan
bingung : bingung
memilih : pilih
uang : uang
saku : saku
terbatas : batas
bulannya : bulan
memasang : pasang
hasil : hasil
penelitian : teliti
keputusan : putus
pemilihan : pilih
metode : metode
ahp : ahp
kriteriakriteria : kriteriakriteria
dibutuhkan : butuh
didapatkan : dapat
kuisioner : kuisioner
tagihan : tagih
ecommerce : ecommerce
fitur : fitur
transfer : transfer
tran

In [100]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

DATA_SI['Abstrak_tokens_stemmed'] = DATA_SI['Abstrak_tokens_WSW'].swifter.apply(get_stemmed_term)

print(DATA_SI['Abstrak_tokens_stemmed'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Pandas Apply:   0%|          | 0/80 [00:00<?, ?it/s]

0     [oktober, total, selenggara, fintech, dukung, ...
1     [tubuh, manusia, ginjal, organ, fungsi, filtra...
2     [taman, hutan, raya, raden, soerjo, tahura, rs...
3     [taman, hutan, raya, raden, soerjo, tahura, rs...
4     [ginjal, korteks, ginjal, medula, ginjal, pelv...
                            ...                        
75    [bem, febtd, unusa, sistem, informasi, bas, we...
76    [pisang, buah, favorit, masyarakat, indonesia,...
77    [akuntasi, regista, bunga, wijaya, cabang, sur...
78    [kelola, data, stok, barang, lapor, toko, opti...
79    [giat, ppdb, giat, wajib, lembaga, didik, port...
Name: Abstrak_tokens_stemmed, Length: 80, dtype: object


In [101]:
#stopwords #2

from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["ada", "hasil", "jadi", "pt", "buat", "ajar", "hari", "tahun", "artikel",
                       "metode", "guna", "the"])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

DATA_SI['Abstrak_tokens_stemmed2'] = DATA_SI['Abstrak_tokens_stemmed'].apply(stopwords_removal)

print(DATA_SI['Abstrak_tokens_stemmed2'].head())

0    [oktober, total, selenggara, fintech, dukung, ...
1    [tubuh, manusia, ginjal, organ, fungsi, filtra...
2    [taman, hutan, raya, raden, soerjo, tahura, rs...
3    [taman, hutan, raya, raden, soerjo, tahura, rs...
4    [ginjal, korteks, ginjal, medula, ginjal, pelv...
Name: Abstrak_tokens_stemmed2, dtype: object


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [102]:
for i in range(len(DATA_SI)):
        a=DATA_SI.iloc[i][6]
        document.append(a)

document[0:5]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['kegiatan', 'ppdb', 'kegiatan', 'wajib', 'lembaga']

In [103]:
doc_clean = DATA_SI['Abstrak_tokens_stemmed2']
doc_clean

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0     [oktober, total, selenggara, fintech, dukung, ...
1     [tubuh, manusia, ginjal, organ, fungsi, filtra...
2     [taman, hutan, raya, raden, soerjo, tahura, rs...
3     [taman, hutan, raya, raden, soerjo, tahura, rs...
4     [ginjal, korteks, ginjal, medula, ginjal, pelv...
                            ...                        
75    [bem, febtd, unusa, sistem, informasi, bas, we...
76    [pisang, buah, favorit, masyarakat, indonesia,...
77    [akuntasi, regista, bunga, wijaya, cabang, sur...
78    [kelola, data, stok, barang, lapor, toko, opti...
79    [giat, ppdb, giat, wajib, lembaga, didik, port...
Name: Abstrak_tokens_stemmed2, Length: 80, dtype: object

**LDA GENSIM**

In [104]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
print(dictionary)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Dictionary<1864 unique tokens: ['ahp', 'alternatif', 'and', 'annie', 'aplikasi']...>


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [105]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 3 # jumlah topik yang akan di extract
number_words = 10 # jumlah kata per topik

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [106]:
# Running and Trainign LDA model on the document term matrix.
lda_model = Lda(doc_term_matrix, num_topics=total_topics, id2word = dictionary, passes=50)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.017*"sistem" + 0.017*"teliti" + 0.015*"milik" + 0.013*"nilai" + 0.012*"use" + 0.012*"kualitas" + 0.010*"variabel" + 0.009*"informasi" + 0.009*"manfaat" + 0.008*"ukur"'),
 (1,
  '0.043*"sistem" + 0.020*"informasi" + 0.017*"teliti" + 0.017*"uji" + 0.016*"data" + 0.011*"butuh" + 0.008*"website" + 0.008*"testing" + 0.008*"mudah" + 0.008*"teknologi"'),
 (2,
  '0.015*"of" + 0.012*"and" + 0.012*"data" + 0.011*"citra" + 0.011*"ginjal" + 0.011*"fitur" + 0.010*"tubulus" + 0.009*"nilai" + 0.009*"proksimal" + 0.007*"in"')]

In [107]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])
print(df_imp_wcount)

         word  topic_id  importance  word_count
0      sistem         0    0.016958         264
1      teliti         0    0.016820         148
2       milik         0    0.015128          82
3       nilai         0    0.013279          72
4         use         0    0.011824          44
5    kualitas         0    0.011696          43
6    variabel         0    0.009783          32
7   informasi         0    0.009314         128
8     manfaat         0    0.008931          36
9        ukur         0    0.007933          28
10     sistem         1    0.043122         264
11  informasi         1    0.020309         128
12     teliti         1    0.017000         148
13        uji         1    0.016575          93
14       data         1    0.015907         121
15      butuh         1    0.011209          65
16    website         1    0.008340          68
17    testing         1    0.008241          45
18      mudah         1    0.007900          42
19  teknologi         1    0.007759     

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [108]:
from google.colab import drive
drive.mount('drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [109]:
#simpan ke google drive
df_imp_wcount.to_csv('df_imp_wcount_unusa.csv')
!cp df_imp_wcount.csv "drive/My Drive/LDA/"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [110]:
#Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=doc_term_matrix, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [111]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))

   Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0               2              0.8371   
1            1               2              0.9941   
2            2               0              0.9959   
3            3               0              0.9959   
4            4               2              0.9918   
5            5               0              0.9951   
6            6               2              0.9963   
7            7               1              0.9926   
8            8               1              0.9925   
9            9               1              0.9928   

                                            Keywords  \
0  of, and, data, citra, ginjal, fitur, tubulus, ...   
1  of, and, data, citra, ginjal, fitur, tubulus, ...   
2  sistem, teliti, milik, nilai, use, kualitas, v...   
3  sistem, teliti, milik, nilai, use, kualitas, v...   
4  of, and, data, citra, ginjal, fitur, tubulus, ...   
5  sistem, teliti, milik, nilai, use, kualitas, v...   
6  of, and, d

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
#simpan ke google drive
df_dominant_topic.to_csv('df_dominant_topic_unusa.csv')
!cp df_dominant_topic.csv "drive/My Drive/LDA/"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**VISUALISASI**

In [113]:
import pyLDAvis.gensim
import pickle
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [114]:
import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(total_topics))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [115]:
corpus = [dictionary.doc2bow(text) for text in doc_clean]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [116]:
# proses
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [117]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [118]:
pyLDAvis.save_html(LDAvis_prepared, 'drive/My Drive/LDA/ldavis_prepared_'+ str(total_topics) +'.html')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [119]:
# proses
LDAvis_prepared

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.139535  0.095341       1        1  50.943151
0      0.044384 -0.135077       2        1  30.867747
2     -0.183919  0.039735       3        1  18.189102, topic_info=          Term        Freq       Total Category  logprob  loglift
59      sistem  227.000000  227.000000  Default  30.0000  30.0000
339         of   31.000000   31.000000  Default  29.0000  29.0000
2          and   27.000000   27.000000  Default  28.0000  28.0000
85      ginjal   17.000000   17.000000  Default  27.0000  27.0000
78       citra   30.000000   30.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
41       nilai   13.390264   57.791984   Topic3  -4.7287   0.2420
76        beda    6.397337   12.062677   Topic3  -5.4674   1.0701
64      teliti    8.887817  124.278958   Topic3  -5.1386  -0.9335
176    website    6.887728   57.338422   Topic3  -5.3935  -0.4149
81   ekstraksi    5.406557   10.582643   Topic3  -5.6357   1.0327

[190 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1086      1  0.923433    ancang
2         2  0.292498       and
2         3  0.694682       and
4         1  0.589314  aplikasi
4         2  0.342182  aplikasi
...     ...       ...       ...
671       1  0.946614       web
176       1  0.610411   website
176       2  0.261605   website
176       3  0.122082   website
380       3  0.865474      with

[230 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])